In [8]:
from collections import namedtuple
import numpy as np
import torch as tr

TrainConfig = namedtuple(
    'TrainConfig',
    'n_step_tboard_log '
    'n_step_console_log '
    'n_step_validation '
    'n_step_save_params '
    'n_step_visualize'
)

In [2]:
class Config:
    use_gpu = tr.cuda.is_available()

    dtype = tr.float32

    base_port = 8001

    default_train_config = TrainConfig(
        n_step_tboard_log=50,
        n_step_console_log=-1,
        n_step_validation=100,
        n_step_save_params=1000,
        n_step_visualize=500
    )
    
x = Config()
x.default_train_config.n_step_console_log

In [28]:
def tensor_output(use_gpu=Config.use_gpu):
    def outer(f):
        def inner(*args,**kwargs):
            ret = f(*args,**kwargs)
            return tr.tensor(ret)
        return inner
    return outer

In [29]:
@tensor_output(use_gpu=Config.use_gpu)
def numpy(val=10):
    return np.random.rand(val)

In [32]:
numpy(1)

tensor([0.5654], dtype=torch.float64)

In [33]:
x = tr.tensor([1])

In [36]:
np.random.permutation(10)

array([7, 4, 5, 2, 9, 1, 3, 6, 0, 8])

In [1]:
import numpy as np
import torch as tr
from torch.nn.parameter import Parameter
from torch.nn import functional as F
from base.hyperparams import Hyperparams
from base.model import BaseModel
from torch import nn
from exp_context import ExperimentContext
from utils.commons import NLinear

In [2]:
class UpConvBlock(nn.Sequential):
    def __init__(self, in_channels, out_channels, bn=True, kernel_size=5, stride=2, output_padding=(0, 0), padding=(2, 2)):
        layers = [
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, output_padding=output_padding,
                               padding=padding)]
        if bn:
            layers.append(nn.BatchNorm2d(out_channels, 0.8))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        super(UpConvBlock, self).__init__(*layers)

In [3]:
tconv1 = UpConvBlock(in_channels=32, out_channels=16, kernel_size=5, stride=2, output_padding=(1, 1))

In [4]:
tconv1

UpConvBlock(
  (0): ConvTranspose2d(32, 16, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (1): BatchNorm2d(16, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.2, inplace=True)
)

In [5]:
from dataloaders.factory import DataLoaderFactory

import numpy as np
import os, argparse, json, logging
import matplotlib.pyplot as plt

import matplotlib
from tqdm import tqdm

In [6]:
matplotlib.use('Agg')

In [7]:
from exp_context import ExperimentContext

In [8]:
default_args_str = '-hp hyperparameters/digit_mnist.py -d all -en awa -t'

In [9]:
# Argument Parsing
parser = argparse.ArgumentParser()

parser.add_argument('-g', '--gpu', default=0, help='index of the gpu to be used. default: 0')
parser.add_argument('-t', '--tensorboard', default=False, const=True, nargs='?', help='Start Tensorboard with the experiment')
parser.add_argument('-r', '--resume', nargs='?', const=True, default=False,
                    help='if present, the training resumes from the latest step, '
                         'for custom step number, provide it as argument value')
parser.add_argument('-d', '--delete', nargs='+', default=[], choices=['logs', 'weights', 'results', 'all'],
                    help='delete the entities')
parser.add_argument('-w', '--weights', nargs='?', default='iter', choices=['iter', 'best_gen', 'best_pred'],
                    help='weight type to load if resume flag is provided. default: iter')
parser.add_argument('-hp', '--hyperparams', required=True, help='hyperparam class to use from HyperparamFactory')
parser.add_argument('-en', '--exp_name', default=None, help='experiment name. if not provided, it is taken from Hyperparams')


_StoreAction(option_strings=['-en', '--exp_name'], dest='exp_name', nargs=None, const=None, default=None, type=None, choices=None, help='experiment name. if not provided, it is taken from Hyperparams', metavar=None)

In [10]:
args = parser.parse_args(default_args_str.split())

In [11]:
print(json.dumps(args.__dict__, indent=2))

{
  "gpu": 0,
  "tensorboard": true,
  "resume": false,
  "delete": [
    "all"
  ],
  "weights": "iter",
  "hyperparams": "hyperparameters/digit_mnist.py",
  "exp_name": "awa"
}


In [12]:
resume_flag = args.resume is not False
ExperimentContext.set_context(args.hyperparams, args.exp_name)
H = ExperimentContext.Hyperparams  # type: Hyperparams
exp_name = ExperimentContext.exp_name

loading HP from file
importing hyperparams hyperparameters.digit_mnist


In [13]:
logger = logging.getLogger(__name__)
LOG_FORMAT = "[{}: %(filename)s: %(lineno)3s] %(levelname)s: %(funcName)s(): %(message)s".format(ExperimentContext.exp_name)
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)


In [14]:
#### Clear Logs and Results based on the argument flags ####
from paths import Paths
from utils import bash_utils, model_utils

In [15]:
if 'all' in args.delete or 'logs' in args.delete or resume_flag is False:
    logger.warning('Deleting Logs...')
    bash_utils.delete_recursive(Paths.logs_base_dir)
    print('')

if 'all' in args.delete or 'results' in args.delete:
    logger.warning('Deleting all results in {}...'.format(Paths.results_base_dir))
    bash_utils.delete_recursive(Paths.results_base_dir)
    print('')


[awa: <ipython-input-15-b5d7d7b2dfb5>:   2] WARNING: <module>(): Deleting Logs...
[awa: <ipython-input-15-b5d7d7b2dfb5>:   7] WARNING: <module>(): Deleting all results in ../experiments/awa/results...


In [16]:
##### Create required directories
model_utils.setup_dirs()
print('dirs created')
###################
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dirs created


In [25]:
dl = DataLoaderFactory.get_dataloader(H.dataloader, H.batch_size, H.batch_size)

<class 'torch.Tensor'>


In [26]:
dl

In [18]:
import base.dataloader as DL

In [20]:
data = DL.get_z_dist(1,'normal',bounds=1)

In [23]:
data.size()

torch.Size([1, 100])

In [34]:
import base.dataloader as DL
from models.mnist.gan import ImgGAN
from dataloaders.mnist import MnistDataLoader
mdl = MnistDataLoader()
train_data, test_data, train_labels, test_labels = mdl.get_data()

<class 'torch.Tensor'>


In [ ]:
epochs = 100
with tqdm(total = epochs) as pbar:
    for ep in range(epochs):
        print(ep)
        # add code from here
        noise = DL.get_z_dist(H.batch_size, 'normal', bounds=1)
        if flag == 0:
            gen_loss = ImgGAN.gen_adv_loss_x(noise)
            gen_loss.backward()
            ImgGAN.opt['generator'].step()
            if(ep%20 == 0)
                flag = 1            
        else:
            for i_batch, x in enumerate(train_data):
                disc_loss = ImgGAN.disc_adv_loss_x(x, noise)
                disc_loss.backward()
                ImgGAN.opt['disc_x'].step()
            count=+1
            if(count == 20):
                flag = 0
        pbar.update(1)